Construct logregs on subfolds and blend , over different random splits to subfolds 

Logregs with "optimized" C for each target

36_4 - blend many C  

V4 - decrease n_blends to 1 from 2 ( number of blends for each C ) 

V3: Extend further C interval -  Your notebook was stopped because it exceeded the max allowed execution duration. (32401 seconds )

V2 : twenty C : list4blend_change_optimal_C_by = 1.15 ** np.arange(-10,10) , Additionally each C point is double blended with different random splits to folds:  got CV 0.01543 - it is around top CV results obtained in that way. That is strange.


v1 - ten C -> C * np.linspace(0.5,1,10)

36_3 - blend three  C



In [ ]:
# Set param:
# Set what C will be blended (relative to previosly chosen optimal C for each target)

n_blends = 1


import numpy as np
#np.linspace(0.5,1,10), 1.2**np.arange(-10,10) # [-2,-1,0,1,2,3,4]

list4blend_change_optimal_C_by = 1.3**np.arange(-12,20)

np.round(list4blend_change_optimal_C_by,2)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# V6 - list_C is min from cv3 rs0,1,100 finer grid than before

list_C = [0.03, 0.003, 0.0002, 0.001, 0.0007, 0.001, 0.001, 0.002, 0.1, 0.005, 0.001, 0.01, 0.5, 0.01, 0.0007, 0.002, 0.001, 0.001, 0.001, 0.001, 0.002, 0.002, 0.007, 0.005, 0.005, 0.01, 0.005, 0.005, 0.003, 0.002, 0.01, 0.005, 0.002, 0.03, 0.0, 0.1, 0.005, 0.05, 0.02, 0.007, 0.003, 0.007, 0.005, 0.001, 0.003, 0.005, 0.02, 0.01, 0.01, 0.003, 0.001, 0.01, 0.02, 0.02, 0.001, 0.003, 0.005, 0.001, 0.007, 0.01, 0.0003, 0.0002, 0.005, 0.007, 0.002, 0.2, 0.005, 0.001, 0.002, 0.005, 0.005, 0.001, 0.002, 0.007, 0.007, 0.007, 0.005, 0.002, 0.003, 0.002, 0.02, 0.003, 0.0, 0.002, 0.005, 0.007, 0.05, 0.005, 0.02, 0.01, 0.05, 0.003, 0.007, 0.003, 0.002, 0.02, 0.005, 0.02, 0.002, 0.0007, 0.0007, 0.01, 0.002, 0.01, 0.002, 0.001, 0.1, 0.03, 0.005, 0.05, 0.02, 0.02, 0.01, 0.001, 0.003, 0.002, 0.01, 0.001, 0.02, 0.02, 0.005, 0.01, 0.003, 0.001, 0.003, 0.03, 0.07, 0.02, 0.002, 0.003, 0.0005, 0.002, 0.02, 0.007, 0.003, 0.005, 0.005, 0.01, 0.003, 0.003, 0.003, 0.01, 0.01, 0.002, 0.002, 0.0003, 0.02, 0.005, 0.01, 0.01, 0.01, 0.003, 0.003, 0.007, 0.007, 0.001, 0.0007, 0.01, 0.003, 0.007, 0.03, 0.003, 0.001, 1.0, 0.002, 0.02, 0.005, 0.0, 0.001, 0.05, 0.007, 0.05, 0.01, 0.03, 0.003, 0.007, 0.0005, 0.002, 0.005, 0.005, 0.002, 0.005, 0.001, 0.007, 0.007, 0.0007, 0.02, 0.005, 0.03, 0.005, 0.007, 0.001, 0.003, 0.005, 0.007, 0.002, 0.001, 0.02, 0.003, 0.01, 0.003, 0.02, 0.01, 0.0005, 0.1, 0.0007]


In [ ]:
for c in list_C[:10]:
    print(c, type(c))
    

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime
import time

import matplotlib.pyplot as plt
import seaborn as sns

import time
df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',index_col = 0)  
df0 = df.copy()
df['cp_type'] = df['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':1.0}) # Forget about control group  
df['cp_dose'] = df['cp_dose'].map({'D1':0.0, 'D2':1.0})
df['cp_time'] = df['cp_time'].map({24:0.0, 48: .5 , 72:1.0})
X = df.copy()
X_save = X.copy()
df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv',index_col = 0)
df0_test = df_test.copy()
df_test['cp_type'] = df_test['cp_type'].map({'trt_cp':1.0, 'ctl_vehicle':0.0})
df_test['cp_dose'] = df_test['cp_dose'].map({'D1':0.0, 'D2':1.0})
df_test['cp_time'] = df_test['cp_time'].map({24:0.0, 48: .5 , 72:1.0})

y = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv',index_col = 0 )
y_save = y.copy()
print(y.iloc[:3,:2])
df

In [ ]:
y_save.sum(axis = 0 ).sort_values(ascending = False)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Simulation core

In [ ]:
from sklearn.linear_model import LogisticRegression


df_submit = pd.DataFrame(index = df_test.index)
df_train_oof_pred = pd.DataFrame(index = df0.index) 

df_stat = pd.DataFrame()
t00 = time.time()
df_stat = pd.DataFrame()
cnt4df_stat = 0
for cnt_target, target_name in enumerate(y_save.columns):#enumerate(['dopamine_receptor_antagonist']) : # y_save.columns):
    C = list_C[ cnt_target ]
    y = y_save[target_name]
    #if target_name == 'dopamine_receptor_antagonist':
    #    C = 0.003
    
    if C != 0:
        y_pred_submit = np.zeros( len(df_test) )
        cnt_blend_submit = 0
        y_pred_oof_blend = np.zeros_like(y,dtype = float)
        cnt_blend_oof = 0
        for C_effective in C*list4blend_change_optimal_C_by: # np.linspace(0.5,1,10): # np.array([0.5,1, 2]):
            model = LogisticRegression( C = C_effective  ) #, penalty='l1', solver = 'liblinear' )  
            for cnt in range(n_blends):
                rs = np.random.randint(10**7)
                skf = StratifiedKFold(n_splits=3, shuffle=True, random_state= rs )
                y_pred_oof = np.zeros_like(y,dtype = float)
                list_loss_train = []
                for train_index, test_index in skf.split(X, y):
                    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                    model.fit(X_train,y_train)
                    y_pred_oof[ test_index ] = model.predict_proba(X_test)[:,1]
                    y_pred_submit = (y_pred_submit*cnt_blend_submit + model.predict_proba(df_test)[:,1])/(cnt_blend_submit+1) # blend
                    cnt_blend_submit += 1

                    list_loss_train = log_loss(y_train,  model.predict_proba(X_train)[:,1])

                y_pred_oof_blend = (y_pred_oof_blend*cnt_blend_oof + y_pred_oof) / (cnt_blend_oof + 1) # blend
                cnt_blend_oof += 1

                df_stat.loc[cnt4df_stat,'Target'] = target_name
                df_stat.loc[cnt4df_stat,'Count Blend'] = cnt_blend_oof
                df_stat.loc[cnt4df_stat,'LogLoss OOF'] = log_loss(y, y_pred_oof )
                df_stat.loc[cnt4df_stat,'LogLoss Blend OOF'] = log_loss(y, y_pred_oof_blend )
                df_stat.loc[cnt4df_stat,'LogLoss Train'] = np.mean( list_loss_train )
                df_stat.loc[cnt4df_stat,'Seconds Passed'] = np.round( time.time() - t00 )
                df_stat.loc[cnt4df_stat,'Random Seed'] = rs
                df_stat.loc[cnt4df_stat,'C'] = C
                cnt4df_stat += 1
        df_train_oof_pred.loc[:,target_name] = y_pred_oof_blend
    else:
        mn = y[df0.cp_type=='trt_cp'].mean()
        print(mn, y.mean(), target_name, y.sum() )
        y_pred_submit = np.ones_like(df_test.iloc[:,0])*mn # y.mean() 
        df_train_oof_pred.loc[:,target_name] = np.ones_like(df0.iloc[:,0])*mn
    df_submit.loc[:,target_name] = y_pred_submit        
    
    print(cnt_target,   target_name, 'Blend', np.round(df_stat.loc[cnt4df_stat-1,'LogLoss Blend OOF'], 5) , 'No blend', np.round(df_stat.loc[cnt4df_stat-1,'LogLoss OOF'] ,5) )
    
total_time = time.time()-t00    
#df_stat.to_csv("df_stat.csv")
print(np.round(total_time,0), np.round(total_time/60,0),np.round(total_time/3600,1),'seconds, minutes, hours total passed')  
df_train_oof_pred.to_csv('df_train_oof_pred.csv')
df_submit



In [ ]:
df_stat.head()

In [ ]:
print(np.round(total_time,0), np.round(total_time/60,0),np.round(total_time/3600,1),'seconds, minutes, hours total passed')  
df_stat.to_csv('df_stat.csv')

In [ ]:
lc = list(  filter(lambda x: 'LogLoss' in x , df_stat.columns) )

name_oof = lc[0]
df_stat2 = pd.DataFrame()
for i,c in enumerate(y_save.columns):
    if c not in list(  df_stat['Target'] ) : continue
    
    m = df_stat['Target'] == c
    df_stat2.loc[c,'Internal Numero'] = i 
    df_stat2.loc[c,'Target Sum'] = y_save[c].sum()
    df_stat2.loc[c,'Blend gain * 1e5'] = np.round( - 1e5*( df_stat[m]['LogLoss Blend OOF'].iat[-1] - df_stat[m]['LogLoss OOF'].mean() ) , 1)
    
    df_stat2.loc[c,'Logloss predict by mean'] = log_loss(y_save[c], np.ones_like(y_save[c])*y_save[c].mean() ) 
    df_stat2.loc[c,'Logloss OOF'] = df_stat[m]['LogLoss OOF'].mean()
    df_stat2.loc[c,'Logloss Blend OOF'] = df_stat[m]['LogLoss Blend OOF'].iat[-1]
    df_stat2.loc[c, 'LogLoss Train'] = df_stat[m][ 'LogLoss Train'].mean()
    
    df_stat2.loc[c,'Logloss Std'] = df_stat[m]['LogLoss OOF'].std()
    df_stat2.loc[c,'Logloss Train Std'] = df_stat[m]['LogLoss Train'].std()
    #df_stat[m]
df_stat2['Internal Numero'] = df_stat2['Internal Numero'].astype(int)    

df_stat2.to_csv('df_stat2_by_targets_aggregated.csv')

df_stat2.sort_values('Blend gain * 1e5', ascending = False)


In [ ]:
(df_stat2['Blend gain * 1e5'] > 0 ).sum() , 'out of 206',

In [ ]:
(df_stat2['Blend gain * 1e5'] > 10 ).sum() , 'out of 206',

In [ ]:
(df_stat2['Blend gain * 1e5'] > 50 ).sum() , 'out of 206',

# Intersting Plot - dependence of score on number of blends, compare with scores without blends

In [ ]:
lc = list(  filter(lambda x: 'LogLoss' in x , df_stat.columns) )
d = df_stat.groupby('Count Blend')[lc].mean()
fig = plt.figure(figsize = (15,6))
fig.add_subplot(1,2,1)
for c in d.columns:
    if 'Train' not in c:
        plt.plot(d[c], '*-', label = c)
plt.legend()
plt.grid()
fig.add_subplot(1,2,2)
for c in d.columns:
    if 'Train' in c:
        plt.plot(d[c], '*-', label = c)
plt.legend()
plt.grid()
plt.show()
#print( d[lc].mean() )
#print(d.tail(1) )
print('No blend:', np.round( d['LogLoss OOF'].mean() , 5 )  , 'Blend', np.round( d.tail(1)['LogLoss Blend OOF'].iat[0], 5) )
print('Blend gain *1e5: ', np.round( (d['LogLoss OOF'].mean() -  d['LogLoss Blend OOF'].iat[-1])*1e5 , 0 ) )
print('Best blend result ',d['LogLoss Blend OOF'].min()  )
print('Best Blend gain *1e5: ', np.round( (d['LogLoss OOF'].mean() -  d['LogLoss Blend OOF'].min() )*1e5 , 0 ) )


In [ ]:
d

In [ ]:
df_stat.describe()

In [ ]:
df_stat

In [ ]:
for f in y_save.columns:
    #df_submit[f] = train_targets.loc[ train_features.cp_type=='trt_cp', f].mean()
    df_submit.loc[ df0_test.cp_type!='trt_cp' ,f] = 0
df_submit#.head()

In [ ]:
df_submit.to_csv("submission.csv")
